<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/MTL/MTL_1head_alternating_diiferent%20loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install wandb

# MTL approach
- Use Soft and Hard Labels (Alternate: Add hard loss every fith batch)
- Hard:
  - Encoding: ~~1_hot~~ / One label 
  - Loss: ~~Cross_entropy~~/ Negative_Log_Like
- Soft
  - Loss: ~~Cross_entropy~~ / KL_div

- Model output layer:
  - ~~1 Layer with 4 outputs (loss with [:2] & [2:])~~
  - ~~2 seperate headers~~
  - 1 header and alternating loss (different loss)

- Layers to train
 - ~~ Whole model~~
 - output
 - Specific number of last layers: 2 (pooler + 11th) 

- Regularization:
  - Dropout



**Results**: 
- https://wandb.ai/capture_disagreement/MTL/runs/4nonuui9?workspace=user-sheuschk

- CodaLab Submission: Alternate_diff_losses_5batches
- CodaLab CE SCore: Average 0.48

In [2]:
#TODO: Fit training to 2 loss functions

In [3]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sheuschk (capture_disagreement). Use `wandb login --relogin` to force relogin


True

In [4]:
from drive.MyDrive.cicl_data.helpers import read_data
# from drive.MyDrive.cicl_data.code import CustomLabelDataset

In [5]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
# from datasets import Dataset
import torch.nn.functional as Fun
from torch.utils.data import Dataset, random_split, DataLoader
from torch.optim import AdamW
import torch.nn as nn

from tqdm.notebook import tqdm


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
data_dict = read_data()
df_all = pd.concat([data_dict[k] for k in data_dict.keys()])

### Pretrained model

In [8]:
# Maybe load from wandb in future
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

In [9]:
from transformers import BertModel
base_model = BertModel.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English")
# class_model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=2)

# output of model: https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

Some weights of the model checkpoint at lanwuwei/GigaBERT-v4-Arabic-and-English were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
class MTLModel(nn.Module):
  def __init__(self, base_model):
    super().__init__()
    self.bert = base_model
    self.linear = nn.Linear(768, 2)
    self.dropout = nn.Dropout(0.1)
    # [8, 240, 768]

  def forward(self, input_ids, attention_mask, token_type_ids):
    """a linear layer on top of the pooled output (https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#bertforsequenceclassification)"""

    x = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = self.dropout(x.pooler_output)
    x = self.linear(x)
    x = Fun.log_softmax(x, dim=1)
    return x

In [11]:
model = MTLModel(base_model).to(device)

In [12]:
# Deactivate weights for backprob
for name, param in model.named_parameters():
  if "bert.embeddings" in name:
    param.requires_grad = False
  for i in range(0, 10):  # BERT Model has 11 Bert Layers
    if str(i) in name:
      param.requires_grad = False


## Prepare Data

In [12]:
# from drive.MyDrive.cicl_data.helpers import CustomLabelDataset
# return: input, hard_labels_1hot, soft_labels, hard_labels

In [13]:
class CustomLabelDataset(Dataset):
    def __init__(self, df_all):
        self.text = list(map(self.tokenize_func, df_all["text"]))
        self.soft_labels = df_all["soft_list"] 
        self.hard_labels = df_all["hard_label"]
        self.hard_labels_1h = Fun.one_hot(torch.tensor(df_all['hard_label'].values))

    def __len__(self):
        return len(self.text)
      
    def tokenize_func(self, text):
        return tokenizer(text, padding="max_length", truncation=True, max_length=240)

    def __getitem__(self, idx):
        input = {"attention_mask": torch.tensor(self.text[idx]["attention_mask"]),
                 "token_type_ids": torch.tensor(self.text[idx]["token_type_ids"]),
                 "input_ids": torch.tensor(self.text[idx]["input_ids"])}
        return input, self.hard_labels_1h[idx], torch.tensor(self.soft_labels[idx]), torch.tensor(self.hard_labels[idx])


In [14]:
# Init dataset
dataset = CustomLabelDataset(df_all)
batch_size = 4

train_size = int(len(dataset) * 0.8)
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True)

eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=batch_size)

## Optimization

In [15]:
# Optimizer
num_epochs = 10

num_training_steps = num_epochs * len(train_dataloader)
optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [16]:
# Loss (both expect log probabilities)
loss_hard = nn.NLLLoss()
loss_soft = nn.KLDivLoss(reduction="batchmean")

## Training

In [17]:
run = wandb.init(
    project="MTL",
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "device": device
        },
      save_code = True,
      tags = ["bert_arabic_english", "mtl", "1_head_alternate", "Train 2+1 last layers", "NLL_and_KL_loss"],
      )
wandb.watch(model, log_freq=100)

[]

In [18]:
# from drive.MyDrive.cicl_data.helpers import ce_eval_func
def ce_eval_func(model, eval_dataloader, eval_size, epsilon=1e-12, device="cuda"):
  model.eval()
  cross_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    soft_labels = batch[2].to(device)

    with torch.no_grad():
      pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    probabilities = torch.softmax(pred, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
    cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))

  return cross_error / eval_size



In [23]:
# Train
# num_epochs = wandb.config.epochs

for e in range(num_epochs):
  model.train()
  loss_batches = 0
  epoch_loss = 0
  epoch_len = len(train_dataloader)

  eval_counter = False

  last_ce = 1

  for i, batch in enumerate(train_dataloader):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    hard_labels, soft_labels = batch[3].to(device), batch[2].to(device)

    # predict
    optimizer.zero_grad()
    pred = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # Loss
    loss = loss_soft(pred, soft_labels)

    if i % 5 == 0:
      loss += loss_hard(pred, hard_labels)

    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # Log
    loss_batches += loss.item()
    epoch_loss += loss.item()

    log_n_batches = 200
    if i % log_n_batches == 0:
      if i != 0:
        print(f"{e+1}: Last {log_n_batches} batches avg loss: {loss_batches/log_n_batches:>7f}  [{i}/{epoch_len}]")
        wandb.log({"train/loss_over_batches": loss_batches/log_n_batches})
        wandb.log({"train/epochs": e})
      loss_batches = 0
  
  epoch_loss /= i  # Not completely correct (Loss per batch but not every batch has same size)
  print(f"Epoch [{e+1}/{num_epochs}] mean loss: {epoch_loss:>6f}")
  wandb.log({"train/epoch_loss": epoch_loss})

  # Eval error
  ce = ce_eval_func(model, eval_dataloader, eval_size, device=device)
  print(f"Epoch [{e+1}/{num_epochs}] Eval CE  : {ce:>6f}")
  wandb.log({"eval/epoch_ce": ce})

  # Stop after Eval CE raises 2 times in a row (Simple early stopping)
  if ce > last_ce:
    if eval_counter is True:
      print("Interrupt: Eval Error is raising")
      break;
    eval_counter = True
  else: #TODO: Only save if its best, not just better, than last
    torch.save(model.parameters, 'model.pt')  # Save current best parameter set
    eval_counter = False
  
  last_ce = ce



1: Last 200 batches avg loss: 0.432674  [200/2086]
1: Last 200 batches avg loss: 0.453070  [400/2086]
1: Last 200 batches avg loss: 0.477457  [600/2086]
1: Last 200 batches avg loss: 0.420227  [800/2086]
1: Last 200 batches avg loss: 0.448324  [1000/2086]
1: Last 200 batches avg loss: 0.428431  [1200/2086]
1: Last 200 batches avg loss: 0.455736  [1400/2086]
1: Last 200 batches avg loss: 0.399563  [1600/2086]
1: Last 200 batches avg loss: 0.439243  [1800/2086]
1: Last 200 batches avg loss: 0.427519  [2000/2086]
Epoch [1/10] mean loss: 0.439625


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [1/10] Eval CE  : 0.553140
2: Last 200 batches avg loss: 0.426241  [200/2086]
2: Last 200 batches avg loss: 0.438104  [400/2086]
2: Last 200 batches avg loss: 0.425766  [600/2086]
2: Last 200 batches avg loss: 0.430931  [800/2086]
2: Last 200 batches avg loss: 0.409896  [1000/2086]
2: Last 200 batches avg loss: 0.379195  [1200/2086]
2: Last 200 batches avg loss: 0.406463  [1400/2086]
2: Last 200 batches avg loss: 0.367897  [1600/2086]
2: Last 200 batches avg loss: 0.469681  [1800/2086]
2: Last 200 batches avg loss: 0.415944  [2000/2086]
Epoch [2/10] mean loss: 0.418863


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [2/10] Eval CE  : 0.541497
3: Last 200 batches avg loss: 0.403678  [200/2086]
3: Last 200 batches avg loss: 0.414579  [400/2086]
3: Last 200 batches avg loss: 0.406656  [600/2086]
3: Last 200 batches avg loss: 0.385985  [800/2086]
3: Last 200 batches avg loss: 0.407725  [1000/2086]
3: Last 200 batches avg loss: 0.393377  [1200/2086]
3: Last 200 batches avg loss: 0.413895  [1400/2086]
3: Last 200 batches avg loss: 0.416373  [1600/2086]
3: Last 200 batches avg loss: 0.387015  [1800/2086]
3: Last 200 batches avg loss: 0.406339  [2000/2086]
Epoch [3/10] mean loss: 0.403265


...

Epoch [8/10] Eval CE  : 0.505421
9: Last 200 batches avg loss: 0.325539  [200/2086]
9: Last 200 batches avg loss: 0.336355  [400/2086]
9: Last 200 batches avg loss: 0.363508  [600/2086]
9: Last 200 batches avg loss: 0.309441  [800/2086]
9: Last 200 batches avg loss: 0.361039  [1000/2086]
9: Last 200 batches avg loss: 0.353895  [1200/2086]
9: Last 200 batches avg loss: 0.336812  [1400/2086]
9: Last 200 batches avg loss: 0.324331  [1600/2086]
9: Last 200 batches avg loss: 0.387008  [1800/2086]
9: Last 200 batches avg loss: 0.328840  [2000/2086]
Epoch [9/10] mean loss: 0.343883


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [9/10] Eval CE  : 0.502680
10: Last 200 batches avg loss: 0.338640  [200/2086]
10: Last 200 batches avg loss: 0.316054  [400/2086]
10: Last 200 batches avg loss: 0.321599  [600/2086]
10: Last 200 batches avg loss: 0.327380  [800/2086]
10: Last 200 batches avg loss: 0.322334  [1000/2086]
10: Last 200 batches avg loss: 0.319023  [1200/2086]
10: Last 200 batches avg loss: 0.368564  [1400/2086]
10: Last 200 batches avg loss: 0.353930  [1600/2086]
10: Last 200 batches avg loss: 0.340127  [1800/2086]
10: Last 200 batches avg loss: 0.324917  [2000/2086]
Epoch [10/10] mean loss: 0.333969


  0%|          | 0/522 [00:00<?, ?it/s]

Epoch [10/10] Eval CE  : 0.502680


Model dependent improvements:


## Evaluation

In [26]:
# initialize Evaluation dataset
data_dict_dev = read_data("dev")
df_dev = pd.concat([data_dict_dev[k] for k in data_dict_dev.keys()])

dev_dataset = CustomLabelDataset(df_dev)
dev_batch_size = 4
dev_size = len(dev_dataset)

dev_dataloader = DataLoader(
    dev_dataset,
    batch_size=dev_batch_size)

In [27]:
# Cross Entropy Error
cross_error = ce_eval_func(model, dev_dataloader, dev_size, device=device)
print(f"CE error: {cross_error}")
wandb.log({"dev/ce": cross_error})

  0%|          | 0/557 [00:00<?, ?it/s]

CE error: 0.48155322670936584


In [28]:
# from drive.MyDrive.cicl_data.helpers import f1_eval_func
from sklearn.metrics import f1_score

def f1_eval_func(model, eval_dataloader, eval_size, device="cuda"):
  model.eval()
  f1_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    hard_labels = batch[3].to(device)

    with torch.no_grad():
      pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    f1_error += f1_score(hard_labels.cpu(), pred.argmax(1).cpu(), average='micro')

  return f1_error / eval_size

In [29]:
# F1 micro Error
f1 = f1_eval_func(model, dev_dataloader, dev_size, device)
print(f"F1 error: {f1}")
wandb.log({"dev/f1": f1})

  0%|          | 0/557 [00:00<?, ?it/s]

F1 error: 0.19561797752808988


In [24]:
model.eval()
cross_error = 0
epsilon = 1e-12
for i, batch in enumerate(eval_dataloader):
  input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
  attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
  token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
  soft_labels = batch[2].to(device)

  with torch.no_grad():
    pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
  probabilities = torch.softmax(pred, axis=-1)
  predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
  cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))
  break



In [25]:
print(predictions)
print(soft_labels)

tensor([[0.8037, 0.1963],
        [0.8281, 0.1719],
        [0.7596, 0.2404],
        [0.8965, 0.1035]], device='cuda:0')
tensor([[1.0000, 0.0000],
        [0.4000, 0.6000],
        [0.6000, 0.4000],
        [1.0000, 0.0000]], device='cuda:0')


### Finish

In [30]:
# torch.save(model.parameters, 'model.pt')
artifact = wandb.Artifact(name='model_param', type='model')
artifact.add_file(local_path="model.pt")
run.log_artifact(artifact);

In [31]:
wandb.finish()

dev/ce,▁
dev/f1,▁
eval/epoch_ce,█▅▄▃▅▂▂▁▁▁▁
train/epoch_loss,█▆▅▅▅▄▃▃▂▂▁
train/epochs,▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
train/loss_over_batches,██▆▇▆▅▆▅▅▃▃▄▄▅▄▅▄▅▄▄▄▄▂▂▃▃▃▂▃▃▃▂▂▃▂▁▂▁▃▁
dev/ce,0.48155
dev/f1,0.19562
eval/epoch_ce,0.50268
train/epoch_loss,0.33397
train/epochs,9


## TSV files

In [32]:
import os
import csv

In [34]:
filepaths = ["/content/ArMIS_results.tsv", "/content/ConvAbuse_results.tsv", "/content/HS-Brexit_results.tsv", "/content/MD-Agreement_results.tsv"]
epsilon = 1e-12

for fp in filepaths:
  if os.path.exists(fp):
    os.remove(fp)

for key in data_dict_dev.keys():
  tsv_dataset = CustomLabelDataset(data_dict_dev[key])
  tsv_dataloader = DataLoader(tsv_dataset, shuffle=False, batch_size=1)
  filepath_write = f"/content/{key}_results.tsv"

  with open(filepath_write, 'w', newline='') as tsvfile:
      writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
      for i, batch in enumerate(tqdm(tsv_dataloader, 0)):
        input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
        attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)

        with torch.no_grad():
          pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # logits = pred.logits
        probability = torch.softmax(pred, axis=-1)
        prediction = torch.argmax(probability, dim=-1)
        probability = torch.clip(probability, epsilon, 1. - epsilon) # Really necessary?
        writer.writerow([prediction[0].item(), probability[0][0].item(), probability[0][1].item()])


  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/1104 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?it/s]

In [35]:
from zipfile import ZipFile

filepath = "res.zip" 

if os.path.exists(filepath):
    os.remove(filepath)

#loop over filepath names throws an string index out of range for whatever reason(also can't use content here, not sure why)
with ZipFile(filepath, 'w') as zipObj:
  zipObj.write("MD-Agreement_results.tsv")
  zipObj.write("ArMIS_results.tsv")
  zipObj.write("HS-Brexit_results.tsv")
  zipObj.write("ConvAbuse_results.tsv")

In [36]:
from google.colab import files
files.download("res.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>